In [63]:
# Importing libraries
import pandas as pd
pd.set_option('display.max_columns', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors


import math

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import igraph as ig

import folium
from folium import PolyLine, CircleMarker
from folium.plugins import MarkerCluster
from shapely.geometry import Point, LineString
from pyvis.network import Network

import community
from community import community_louvain
import matplotlib.cm as cm
import leidenalg


In [64]:
### 1) Prepare dataset for Graph building

# Loading dataset

df_final = pd.read_csv("/Users/jangaydoul/Desktop/Copenhagen Business School/4. Semester :: Thesis/03_Data/csvs/df_final.csv")
coordinates_df = pd.read_csv("/Users/jangaydoul/Desktop/Copenhagen Business School/4. Semester :: Thesis/03_Data/csvs/df_graph.csv")

In [65]:
print(len(coordinates_df)) #162,112
print(len(df_final)) #4,973,604

162112
4973604


In [66]:
df_final['FromLocation'].nunique() #37,994
df_final['ToLocation'].nunique() #87,093

87093

In [67]:
# Group the data by pick-up and drop-off points
grouped = df_final.groupby(['FromLatitude', 'FromLongitude', 'ToLatitude', 'ToLongitude'])

# Count the number of unique groups
unique_routes = len(grouped)

# Print the result
print("Number of unique routes:", unique_routes)

Number of unique routes: 162113


In [68]:
# Drop duplicates from df_final based on the four coordinate variables
df_final_unique = df_final.drop_duplicates(subset=['FromLatitude', 'FromLongitude', 'ToLatitude', 'ToLongitude'])

# Select the columns to merge from df_final
columns_to_merge = ['FromLatitude', 'FromLongitude', 'ToLatitude', 'ToLongitude',
                    'FromLocation', 'ToLocation', 'FromCity', 'ToCity', 'FromCountry', 'ToCountry', 'FullLoadIndicator']

# Merge the two dataframes on the four coordinate variables
df_graph = pd.merge(coordinates_df, df_final_unique[columns_to_merge], on=['FromLatitude', 'FromLongitude', 'ToLatitude', 'ToLongitude'])


In [69]:
# Count the frequency of each unique route in df_final
route_frequency = df_final.groupby(['FromLatitude', 'FromLongitude', 'ToLatitude', 'ToLongitude']).size().reset_index(name='route_count')

# Merge the frequency information with merged_df
df_graph = pd.merge(df_graph, route_frequency, on=['FromLatitude', 'FromLongitude', 'ToLatitude', 'ToLongitude'])


In [70]:
df_graph.head()

,FromLatitude,FromLongitude,ToLatitude,ToLongitude,route_distance,FromLocation,ToLocation,FromCity,ToCity,FromCountry,ToCountry,FullLoadIndicator,route_count
0,50.63656,4.78251,63.44178,10.40893,2004.311,17287,42342,Perwez,Trondheim,Belgium,Norway,1.0,17
1,56.20318,14.87340,57.42278,15.06204,160.384,14689,67460,Karlshamn,Vetlanda,Sweden,Sweden,0.0,3
2,57.73572,11.98862,56.16157,15.58364,376.025,25161,27919,Göteborg,Karlskrona,Sweden,Sweden,0.0,5
3,60.71076,10.60526,51.00683,-0.42949,1991.237,21503,28913,Raufoss,BILLINGSHURST,Norway,United Kingdom,1.0,2
4,57.73572,11.98862,56.21754,15.27024,348.243,25161,30402,Göteborg,Ronneby,Sweden,Sweden,0.0,2


In [71]:
df_graph['route_count'].sum()

4973250

In [72]:
len(df_graph)

162112

In [13]:
### 1.1) Filtering for Country (UK in this case) and distance < 300km 

df_graph_UK = df_graph[(df_graph['FromCountry'] == 'United Kingdom') & (df_graph['ToCountry'] == 'United Kingdom')]
df_graph_UK_300 = df_graph_UK[df_graph_UK['route_distance'] < 300]

len(df_graph_UK_300)

19720

In [ ]:
df_graph_UK_300['route_distance'].describe()

In [73]:
### 2) Building Graph

### BASELINE FOR FURTHER CODING

# Build the empty directed graph
G = nx.DiGraph()

# Filter the top 5% most often used routes and add them as edges to the graph
top_routes = df_graph_UK_300.nlargest(int(len(df_graph_UK_300) * 0.05), 'route_count')

for idx, row in top_routes.iterrows():
    G.add_edge((row['FromLatitude'], row['FromLongitude']),
               (row['ToLatitude'], row['ToLongitude']),
               route_count=row['route_count'])

# Calculate the traffic for each node
node_traffic = {}
for edge in G.edges(data=True):
    start, end, data = edge
    route_count = data['route_count']
    
    if start not in node_traffic:
        node_traffic[start] = 0
    if end not in node_traffic:
        node_traffic[end] = 0
    
    node_traffic[start] += route_count
    node_traffic[end] += route_count

# Normalize the node traffic values
max_node_traffic = max(node_traffic.values())
node_traffic_normalized = {node: math.log(traffic + 1) / math.log(max_node_traffic + 1) for node, traffic in node_traffic.items()}

def add_nodes_edges_to_map(graph, folium_map, node_traffic_normalized):
    max_route_count = max(data['route_count'] for _, _, data in graph.edges(data=True))
    
    for edge in graph.edges(data=True):
        start, end, data = edge
        
        # Set the color intensity based on the logarithmic scale of the node traffic
        start_intensity = node_traffic_normalized[start]
        end_intensity = node_traffic_normalized[end]
        
        # Get the color from the color gradient
        start_color = plt.cm.get_cmap('Blues')(start_intensity)
        end_color = plt.cm.get_cmap('Blues')(end_intensity)
        
        start_color = matplotlib.colors.to_hex(start_color)
        end_color = matplotlib.colors.to_hex(end_color)
        
        start_node_marker = CircleMarker(location=start, radius=0.5, color=start_color, fill=True, fill_opacity=0.5)
        end_node_marker = CircleMarker(location=end, radius=0.5, color=end_color, fill=True, fill_opacity=0.5)
        
        intensity = math.log(data['route_count'] + 1) / math.log(max_route_count + 1)
        edge_color = plt.cm.get_cmap('coolwarm')(intensity)
        edge_color = matplotlib.colors.to_hex(edge_color)
        
        polyline = PolyLine(locations=[start, end], color=edge_color, weight=1)
        
        folium_map.add_child(start_node_marker)
        folium_map.add_child(end_node_marker)
        folium_map.add_child(polyline)

# Create the folium map centered based on the coordinates and add nodes and edges to the map
map_center = df_graph_UK_300[['FromLatitude', 'FromLongitude']].mean().tolist()
map_UK = folium.Map(location=map_center, zoom_start=5, tiles='CartoDB Positron')
add_nodes_edges_to_map(G, map_UK, node_traffic_normalized)

# Save the map 
map_UK.save('UK_map.html')

In [74]:
# Counting number of nodes and edges

num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 780
Number of edges: 986


In [75]:
import community.community_louvain as community_louvain

In [81]:
### 3) Detecting Communities

import community.community_louvain as community_louvain

# Convert directed graph to undirected graph for community detection
G_undirected = G.to_undirected()

# Run the Louvain algorithm to identify communities with a resolution of 0.75
partition = community_louvain.best_partition(G_undirected, resolution=0.75)

# Get the number of communities
num_communities = len(set(partition.values()))
print(f"Number of communities: {num_communities}")

# Find the size of each community
community_sizes = {community_id: sum(1 for node in G.nodes() if partition[node] == community_id) for community_id in set(partition.values())}

# Sort communities by size and take the top four
top_communities = sorted(community_sizes.keys(), key=lambda x: community_sizes[x], reverse=True)[:4]

# Assign colors to nodes based on their community
node_colors = []
for node in G.nodes():
    community_id = partition[node]
    if community_id in top_communities:
        if community_id == top_communities[0]:
            color = 'red'
        elif community_id == top_communities[1]:
            color = 'blue'
        elif community_id == top_communities[2]:
            color = 'green'
        else:
            color = 'yellow'
    else:
        color = 'gray'
    node_colors.append(color)

# Create the Folium map and add nodes with community colors from the top four communities
map_UK = folium.Map(location=map_center, zoom_start=5, tiles='CartoDB Positron')
for node, color, community_id in zip(G.nodes(), node_colors, partition.values()):
    if community_id in top_communities:
        marker = CircleMarker(location=node, radius=5, color=color, fill=True, fill_opacity=0.5)
        map_UK.add_child(marker)

# Save the map
map_UK.save('UK_map_with_top_communities.html')


Number of communities: 40


In [82]:
### CHECK

# Compute the modularity of each community
modularity_dict = {}
for node, community_id in partition.items():
    if community_id not in modularity_dict:
        modularity_dict[community_id] = 0
    modularity_dict[community_id] += G.degree(node) - sum(G.degree(n) for n in G.neighbors(node)) / 2.0

# Print the modularity of each community
for community_id, modularity in modularity_dict.items():
    print(f"Community {community_id}: modularity {modularity:.3f}")

Community 1: modularity -691.000
Community 28: modularity 3.000
Community 16: modularity -10.500
Community 6: modularity -359.500
Community 9: modularity -35.500
Community 10: modularity 2.000
Community 27: modularity -712.000
Community 14: modularity -296.500
Community 18: modularity 20.500
Community 20: modularity 5.000
Community 29: modularity 5.000
Community 4: modularity -19.500
Community 0: modularity 5.500
Community 2: modularity 11.500
Community 3: modularity 6.000
Community 5: modularity 1.500
Community 7: modularity 2.500
Community 8: modularity -1.000
Community 17: modularity 9.500
Community 11: modularity 1.500
Community 12: modularity 1.500
Community 13: modularity 1.500
Community 15: modularity 2.000
Community 19: modularity 1.500
Community 21: modularity 1.500
Community 22: modularity 1.500
Community 23: modularity 1.500
Community 24: modularity 1.500
Community 25: modularity 2.000
Community 26: modularity 1.500
Community 30: modularity 1.500
Community 31: modularity 1.5

In [77]:
# Find out most important nodes

betweenness_centrality = nx.betweenness_centrality(G)

# Find the top three most important nodes within each of the top four communities
community_important_nodes = {}
for community_id in top_communities:
    community_nodes = [node for node, c_id in partition.items() if c_id == community_id]
    community_betweenness = {node: betweenness_centrality[node] for node in community_nodes}
    most_important_nodes = sorted(community_betweenness, key=community_betweenness.get, reverse=True)[:3]
    community_important_nodes[community_id] = most_important_nodes

print(community_important_nodes)

{12: [(53.57513, -0.11671), (53.58026, -0.06791), (53.57561, -0.06798)], 1: [(55.72639, -3.95987), (53.58152, -0.06141), (57.12226, -2.08894)], 17: [(52.64919, 0.15457), (52.93645, -1.16425), (53.50446, -2.84867)], 7: [(52.55913, -0.17384), (52.53329, -0.30444), (52.35508, -1.16128)]}


In [78]:
# Visualize top nodes for each community on a map

# Function to get the color for a community
def get_color(community_id):
    if community_id == top_communities[0]:
        return 'red'
    elif community_id == top_communities[1]:
        return 'blue'
    elif community_id == top_communities[2]:
        return 'green'
    else:
        return 'yellow'

# Create the Folium map
map_UK_important_nodes = folium.Map(location=map_center, zoom_start=5, tiles='CartoDB Positron')

# Add the top three most important nodes within each of the top four communities to the map
for community_id, important_nodes in community_important_nodes.items():
    color = get_color(community_id)
    for node in important_nodes:
        marker = CircleMarker(location=node, radius=5, color=color, fill=True, fill_opacity=0.5)
        map_UK_important_nodes.add_child(marker)

# Save the map
map_UK_important_nodes.save('UK_map_important_nodes.html')

In [84]:
print(community_important_nodes)

{12: [(53.57513, -0.11671), (53.58026, -0.06791), (53.57561, -0.06798)], 1: [(55.72639, -3.95987), (53.58152, -0.06141), (57.12226, -2.08894)], 17: [(52.64919, 0.15457), (52.93645, -1.16425), (53.50446, -2.84867)], 7: [(52.55913, -0.17384), (52.53329, -0.30444), (52.35508, -1.16128)]}


In [86]:
### Visualize which routes would be electrified if charging stations were to be placed at most important nodes

# Create a set of all the nodes that are important
important_nodes = set(node for nodes in community_important_nodes.values() for node in nodes)

# Create a set of all the edges that are attached to important nodes
important_edges = set()
for edge in G.edges():
    if edge[0] in important_nodes or edge[1] in important_nodes:
        important_edges.add(edge)

# Create the Folium map
map_UK_important_routes = folium.Map(location=map_center, zoom_start=5, tiles='CartoDB Positron')

# Add the edges to the map
for edge in G.edges(data=True):
    start, end, data = edge
    
    # Set the color of the edge based on whether it's attached to an important node
    edge_color = '#d3d3d3'  # Light gray by default
    if (start, end) in important_edges or (end, start) in important_edges:
        edge_color = '#4169e1'  # Blue for important edges
    
    polyline = PolyLine(locations=[start, end], color=edge_color, weight=1)
    map_UK_important_routes.add_child(polyline)

# Add the nodes to the map
for node in G.nodes():
    # Set the color of the node based on whether it's an important node
    node_color = '#f0f8ff'  # Very light blue by default
    if node in important_nodes:
        node_color = '#00008b'  # Dark blue for important nodes
    
    marker = CircleMarker(location=node, radius=0.5, color=node_color, fill=True, fill_opacity=0.5)
    map_UK_important_routes.add_child(marker)

# Save the map
map_UK_important_routes.save('UK_map_important_routes.html')



In [91]:
### Calculate how much of the network would be electrified if charging stations were being out up at important nodes 
## --> Assumption all incoming and outgoing traffic would be electrfied if charging stations are at node

# Calculating total number of routes and storing as int value 
total_routes = len(df_graph_UK_300)

# Creating new graph 'G_electrified' that only contains the edges tha tare connected to important nodes 
G_electrified = G.edge_subgraph(edge for edge in G.edges() if edge[0] in important_nodes or edge[1] in important_nodes)

# Count number of edges of 'G_electrified' and divide by total number of routes 
electrified_routes = len(G_electrified.edges())
proportion_electrified = electrified_routes / total_routes

# Proportion of deliveries electrified: Sum weights of edges in 'G_electrified' and divide by weights of all edges in the network

# Sum up the weights of the edges in G_electrified
total_electrified_deliveries = sum(data['route_count'] for start, end, data in G_electrified.edges(data=True))

# Sum up the weights of all edges in the network
total_deliveries = sum(data['route_count'] for start, end, data in G.edges(data=True))

# Calculate the proportion of deliveries that would be electrified
proportion_deliveries_electrified = total_electrified_deliveries / total_deliveries

# Print result 
print(f"If charging stations are put at the identified nodes, {proportion_electrified:.2%} of the routes and {proportion_deliveries_electrified:.2%} of the deliveries could be handled by eTrucks")


If charging stations are put at the identified nodes, 2.19% of the routes and 62.21% of the deliveries could be handled by eTrucks
